## Introduction to Data Science

### Data Science Tasks: Classification and Filtering 

In [1]:
import os
import sys
import re
import math
import feedparser
import time
import string
import datetime
import sqlite3
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

Specifying the path to the files:

In [2]:
outputs = "../outputs/"

dbfile1 = "treino.sqlite"
dbfile2 = "treinoblogs.sqlite"
outblog = "blogoutputrss.xml"

db_teste = os.path.join(outputs,dbfile1)
db_blog = os.path.join(outputs,dbfile2)
rssblogoutput = os.path.join(outputs,outblog)

stopwords = nltk.corpus.stopwords.words('portuguese') + nltk.corpus.stopwords.words('english')

#### First block of functions: feature extraction:

In [3]:
def getwords(html):
    '''Remove the HTML tags and cleans the feeds files;splits the sentences 
    by the non alpha characters and converts all words to lowercase.
    Ignores bigger and too small words'''
    #splitter = re.compile('\\W*', flags=re.U)
    #splitter=re.compile(r'[^A-Z^a-z]+', flags=re.U)
    #words=[s.lower() for s in splitter.split(html) if len(s)>2 and len(s)<20]
    words = BeautifulSoup(html, "lxml").findAll(text=True)[0].split(' ')
    words = [w.strip(string.punctuation) for w in words if len(w)>2 and len(w)<20 and w not in stopwords]
    return dict([(w,1) for w in words])

In [4]:
def entryfeatures(entry):
    '''Used when our features are not documents, but feeds rss'''
    splitter=re.compile(r'\W*')
    f={}
    # Extract title words
    titlewords=[s.lower() for s in splitter.split(entry['title']) if len(s)>2 and len(s)<20 and s not in stopwords]
    for w in titlewords: f['Title:'+w]=1
    # Extract summary words
    summarywords=[s.lower() for s in splitter.split(entry['summary']) if len(s)>2 and len(s)<20 and s not in stopwords]
    # Count lowercase words
    uc=0
    for i in range(len(summarywords)):
        w=summarywords[i]
        f[w]=1
        if w.isupper(): uc+=1
        # Features are words in the feed summary
        if i < len(summarywords)-1:
            twowords=' '.join(summarywords[i:i+1])
            f[twowords]=1
    # Save publisher information
    f['Publisher:'+entry['publisher']]=1
    # Too many uppercase words are indicated
    if (float(uc)/(len(summarywords)+1))>0.3:
        f['MAIUSCULAS']=1
    return f

#### Second block of functions: classification

In [5]:
class classifier:
    ''' Represents the classifier, storing what's learnt from training.
    fc saves the combination words/categories {'word': {'bad': 3, 'good': 2}}
    and cc is a dictionary that stores the number of times a category was used
    {'bad': 3, 'good': 2}. Will be used when no DB is used.
    Getfeatures is the feature extraction function to be used'''
    def __init__(self, getfeatures, filename=None, usedb=False):
        self.fc={}
        self.cc={}
        self.getfeatures = getfeatures
        self.usedb = usedb
    
    def setdb(self,dbfile):
        '''When using a database and not dictionaries, to persist the information
        across sessions'''
        self.con = sqlite3.dbapi2.connect(dbfile)    
        self.con.execute('CREATE TABLE IF NOT EXISTS fc(feature,category,count)')
        self.con.execute('CREATE TABLE IF NOT EXISTS cc(category,count)')

    def fcount(self,f,cat):
        '''Returns the number of times a feature appears in a category'''
        if not self.usedb:
            if f in self.fc and cat in self.fc[f]: 
                return float(self.fc[f][cat])
            else: 
                return 0
        else:
            query = 'SELECT COUNT(*) FROM fc WHERE feature=? AND category=?'
            res = self.con.execute(query,(f,cat)).fetchone()
            if res == None: 
                return 0
            else: 
                return float(res[0])

    def incf(self,f,cat):
        '''Creates a feature/category pair if not exists, or increase the number
        if feature exists in a category'''
        if not self.usedb:
            self.fc.setdefault(f,{})
            self.fc[f].setdefault(cat,0)
            self.fc[f][cat] += 1
        else:
            count=self.fcount(f,cat)
            if count == 0:
                self.con.execute('INSERT INTO fc VALUES (?,?,?)',(f,cat,1))
            else:
                query = 'UPDATE fc SET COUNT=? WHERE feature=? AND category=?'
                self.con.execute(query,(count+1,f,cat)) 

    def incc(self,cat):
        '''Increases the number of occurrences of a category'''
        if not self.usedb:
            self.cc.setdefault(cat,0)
            self.cc[cat] += 1        
        else:
            count=self.catcount(cat)
            if count == 0:
                self.con.execute('INSERT INTO cc VALUES (?,?)',(cat,1))
            else:
                query = 'UPDATE cc SET count=? WHERE category=?'
                self.con.execute(query,(count+1,cat))    

    def catcount(self,cat):
        '''Counts the numer of itens in a category'''
        if not self.usedb:
            if cat in self.cc:
                return float(self.cc[cat])
            else:
                return 0
        else:
            query = "SELECT COUNT(*) FROM cc WHERE category=?"
            res=self.con.execute(query,(cat,)).fetchone()
            if res == None:
                return 0
            else:
                return float(res[0])

    def categories(self):
        '''Lists all the categories'''        
        if not self.usedb: return self.cc.keys()
        else:
            cur=self.con.execute('SELECT category FROM cc');
            return [d[0] for d in cur]

    def totalcount(self):
        ''' Returns the total numer of itens'''
        if not self.usedb: return sum(self.cc.values())
        else:
            res=self.con.execute('SELECT SUM(count) FROM cc').fetchone();
            if res==None: return 0
            else: return res[0]

    def train(self,item,cat):
        '''Receives an item (a bag of features) and a category, and increases
        the relative number of this category for all the features'''
        features=self.getfeatures(item)
        for f in features:
            self.incf(f,cat)
        self.incc(cat)
        if self.usedb: self.con.commit()

    def fprob(self,f,cat):
        '''Calculates the probability of a feature to be within a category'''
        if self.catcount(cat)== 0:
            return 0
        return self.fcount(f,cat)/float(self.catcount(cat))

    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        '''Calculates the probability of a feature to appear in a certain category
        as fprob does, but assuming an initial value and changing according to 
        the training. That minimizes the effect of a rare word to be classified
        erroneously'''
        basicprob=prf(f,cat)
        totals=sum([self.fcount(f,c) for c in self.categories()])
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp

In [6]:
class naivebayes(classifier):
    '''Extends classifier class overriding __init__ and adding specific functions
    to classify documents using naive bayes'''
    
    def __init__(self, getfeatures, usedb=False):
        classifier.__init__(self,getfeatures)
        self.thresholds = {}
        self.usedb = usedb
        
    def docprob(self,item,cat):
        '''Calculates the probability of a document to be within a given
        category multiplying all the features probabilities to be in this category'''
        features=self.getfeatures(item)
        p=1
        for f in features: 
            p*=self.weightedprob(f,cat,self.fprob)
        return p

    def prob(self,item,cat):
        catprob=self.catcount(cat)/float(self.totalcount())
        docprob=self.docprob(item,cat)
        return docprob*catprob

    def setthreshold(self,cat,t):
        self.thresholds[cat]=t

    def getthreshold(self,cat):
        if cat not in self.thresholds: 
            return 1.0
        return self.thresholds[cat]

    def classify(self, item, default=None):
        '''Finds the most probably category to be set, and apply this
        classification, given that it satisfies a minimum threshold, compared
        to the second best category to classify; otherwise sets to "None"'''        
        probs = {}
        maximum = 0.0
        #best = None
        for cat in self.categories():
            probs[cat] = self.prob(item, cat)
            if probs[cat] > maximum: 
                maximum = probs[cat]
                best = cat
        for cat in probs:
            if cat == best:
                continue
            if probs[cat]*self.getthreshold(best) > probs[best]: 
                return default
        return best

In [7]:
class fisherclassifier(classifier):
    '''Extends classifier class overriding __init__ and adding specific functions
    to classify documents using fisher method'''

    def __init__(self,getfeatures, usedb=False):
        classifier.__init__(self, getfeatures)
        self.minimums = {}
        self.usedb = usedb
        
    def cprob(self,f,cat):
        '''Returns the frequency of the feature in a category divided
        by the frequency in all categories'''
        clf=self.fprob(f,cat)
        if clf == 0:
            return 0
        freqsum=sum([self.fprob(f,c) for c in self.categories()])
        p=float(clf)/(freqsum)
        return p

    def invchi2(self,chi, df):
        m = chi / 2.0
        sum = term = math.exp(-m)
        for i in range(1, df//2):
            term *= m / i
            sum += term
        return min(sum, 1.0)

    def prob(self,item,cat):
        '''Multipy all the categories, applies the natural log
        and uses the inverse chi2 to calculate probabilty'''
        p = 1
        features = self.getfeatures(item)
        for f in features:
            p *= (self.weightedprob(f,cat,self.cprob))
        fscore = -2*math.log(p)
        return self.invchi2(fscore,len(features)*2)

    def setminimum(self,cat, minimum):
        self.minimums[cat] = minimum

    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]

    def classify(self, item, default=None):
        '''Applies fisher to all categories to find the best result, given 
        that it satisfies a minimum threshold, otherwise sets to "None"'''
        best = default
        maximum = 0.0
        for c in self.categories():
            p = self.prob(item,c)
            if p>self.getminimum(c) and p > maximum:
                best = c
                maximum = p
        return best

#### Third block of functions: reading files or searching for feeds

In [8]:
def blogread(url, classifier):
    '''Receives an url. Tries to classify the entries'''
    f = feedparser.parse(url)
    for entry in f['entries']:
        print('\n-----')
        if 'title' in entry:
            print('Title:     '+ entry['title'])
        if 'publisher' in entry:
            print('Publisher: '+ entry['publisher'])
        else:
            entry['publisher'] = 'Unknown'
        if 'updated_parsed' in entry:
            print('Date Published: ',datetime.datetime.fromtimestamp(time.mktime(entry['updated_parsed'])))
        print('\n-----')
        if 'summary' in entry:
            print(entry['summary'])
        guess = classifier.classify(entry)
        print('\nSuggested: {}'.format(guess))
        cl = input('Enter category or press <enter> to accept suggestion: ').lower()
        if cl == '':
            cl = guess
        if 'title' in entry:
            txt = 'Title:     '+entry['title']
        if 'publisher' in entry:
            txt = txt+'\n'+'Publisher: '+entry['publisher']
        if 'summary' in entry:
            txt = txt+'\n'+entry['summary']
        txt = txt+'\n'+'Suggested: {}'.format(guess)
        if cl == ''.strip() and guess:
            cl = guess
        print('Category "{}" chosen'.format(cl))
        classifier.train(entry,cl)

#### Fourth block of functions: instantiating and training classifiers

In [10]:
def sampletrain(cl):
    print('Running sampletrain to train the classifier...')
    cl.train('Nobody owns the water.','good')
    cl.train('the quick rabbit jumps fences','good')
    cl.train('buy pharmaceuticals now','bad')
    cl.train('make quick money at the online casino','bad')
    cl.train('the quick brown fox jumps','good')

In [11]:
def probabilidades_palavras():
    cl = classifier(getwords)
    print('\n')    
    sampletrain(cl)
    
    print('How many times "quick" --> "good": {}'.format(cl.fcount('quick','good')))
    print('How many times "quick" --> "bad": {}'.format(cl.fcount('quick','bad')))
    print('\nProbability of "quick" given that "good": {}'.format(cl.fprob('quick','good')))
    print('Probability of "money" given that "good" (fprob): {}'.format(cl.fprob('money','good')))
    print('Weighted probability of "money" given that "good" (weightedprob): {}'.format(cl.weightedprob('money','good',cl.fprob)))

    print('\nTraining again with the same documents...\n')
    sampletrain(cl)

    print('\nProbability of "money" given that "good" (fprob): {}'.format(cl.fprob('money','good')))
    print('Weighted probability of "money" given that "good" (weightedprob): {}\n'.format(cl.weightedprob('money','good',cl.fprob)))

In [12]:
probabilidades_palavras()



Running sampletrain to train the classifier...
How many times "quick" --> "good": 2.0
How many times "quick" --> "bad": 1.0

Probability of "quick" given that "good": 0.6666666666666666
Probability of "money" given that "good" (fprob): 0.0
Weighted probability of "money" given that "good" (weightedprob): 0.25

Training again with the same documents...

Running sampletrain to train the classifier...

Probability of "money" given that "good" (fprob): 0.0
Weighted probability of "money" given that "good" (weightedprob): 0.16666666666666666



In [12]:
def probabilidades_documentos_bayes():
    cl = naivebayes(getwords)
    print('\n')    
    sampletrain(cl)
    
    print('Classifying "quick rabbit": {}'.format(cl.classify('quick rabbit', default='unknown')))
    print('Classifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))
    
    print('\nSetting the threshold up...')
    cl.setthreshold('bad',3.0)

    print('Classifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))
    
    print('\nTraining again with the same documents (10x)...')
    for i in range(10): 
        sampletrain(cl)
    
    print('\nClassifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))

In [13]:
probabilidades_documentos_bayes()



Running sampletrain to train the classifier...
Classifying "quick rabbit": good
Classifying "quick money": bad

Setting the threshold up...
Classifying "quick money": unknown

Training again with the same documents (10x)...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...

Classifying "quick money": bad


In [14]:
def probabilidades_palavras_fisher():
    cl = fisherclassifier(getwords)
    print('\n')    
    sampletrain(cl)
    print('\n')      
    print('Probability of "quick" given that "good": {}'.format(cl.cprob('quick', 'good')))
    print('Probability of "money" given that "bad": {}'.format(cl.cprob('money', 'bad')))
    print('Weighted probability of  "money" given that "bad": {}'.format(cl.weightedprob('money','bad',cl.cprob)))

In [15]:
probabilidades_palavras_fisher()



Running sampletrain to train the classifier...


Probability of "quick" given that "good": 0.5714285714285715
Probability of "money" given that "bad": 1.0
Weighted probability of  "money" given that "bad": 0.75


In [16]:
def probabilidades_documentos_fisher():
    cl = fisherclassifier(getwords)
    print('\n')    
    sampletrain(cl)

    print('Classifying "quick rabbit": {}'.format(cl.classify('quick rabbit')))
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))
   
    print('\nSetting the threshold up...')
    cl.setminimum('bad',0.8)
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))

    print('\nSetting the threshold down...')
    cl.setminimum('bad',0.4)
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))

In [17]:
probabilidades_documentos_fisher()



Running sampletrain to train the classifier...
Classifying "quick rabbit": good
Classifying "quick money": bad

Setting the threshold up...
Classifying "quick money": good

Setting the threshold down...
Classifying "quick money": bad


In [13]:
def using_db_example():
    '''Training with a classifier, persisting in a database
    using the training data to classify using another classifier'''
    print('\nInstantiating a fisher classifier...')
    cl = fisherclassifier(getwords, usedb=True)
    cl.setdb(db_teste)
    sampletrain(cl)
    print('\nInstantiating a naive bayes classifier...')    
    cl2 = naivebayes(getwords, usedb=True)
    cl2.setdb(db_teste)
    print('Classifying "quick money": {}'.format(cl2.classify('quick money')))

In [14]:
using_db_example()


Instantiating a fisher classifier...
Running sampletrain to train the classifier...

Instantiating a naive bayes classifier...
Classifying "quick money": bad


In [15]:
def classifying_blogs(url):
    '''Instantiating a new classifier using "entryfeatures" (for feeds)
    Creating the database for the persistance of training data
    Using blogread with searching feeds option - no file reading'''
    print('\nList of categories stored in the database:')
    cl = fisherclassifier(entryfeatures, usedb=True)
    cl.setdb(db_blog)
    for category in cl.categories(): 
        print(category)
    blogread(url, cl) 
    print('\nList of categories stored in the database:')
    for category in cl.categories(): 
        print(category)
    return cl

In [16]:
url = 'http://g1.globo.com/dynamo/rss2.xml'
#url = 'http://feeds.folha.uol.com.br/ciencia/rss091.xml'

In [17]:
cl = classifying_blogs(url)


List of categories stored in the database:

-----
Title:     Dono de posto de combustível em Cuiabá é detido por aumento abusivo durante greve dos caminhoneiros

-----
<img src="https://s2.glbimg.com/ldiYqzcL9AblLqMxvACk-QRdLYM=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/6/f/PXR9TaSkywduYaE3ax5Q/postoautuado.jpg" /><br />   De acordo com a Polícia Civil, a margem de lucro em cima do etanol era de 62%. Fiscalização também flagrou outro posto vendendo combustível com lucro de 70%. Dono de posto em Cuiabá estava lucrando mais de 60% durante greve
Polícia Civil/Divulgação
O dono de um posto de Cuiabá foi detido em flagrante nessa segunda-feira (28) por aumento abusivo do preço de combustível. A detenção ocorreu durante fiscalização da Delegacia Especializada do Consumidor e do Procon Estadual.
O posto do qual ele é proprietário fica na Avenida Miguel Sutil, no Bairro Bosque da Saúde, tinha uma fila enorme para abastecer e vendia o litro da gasolina a 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: FutureWarning: split() requires a non-empty pattern match.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: FutureWarning: split() requires a non-empty pattern match.
  if __name__ == '__main__':


Category "greve" chosen

-----
Title:     VÍDEOS: GRTV 1ª Edição de terça-feira, 29 de maio

-----
<img src="https://s2.glbimg.com/4FZBnNlHcikkaPJqzRvqZFvssL4=/s01.video.glbimg.com/deo/vi/57/05/6770557" /><br />   Confira os vídeos do telejornal com as notícias do sertão de Pernambuco. Confira os vídeos do telejornal com as notícias do sertão de Pernambuco.
Suggested: greve
Enter category or press <enter> to accept suggestion: Estados
Category "estados" chosen

-----
Title:     Feriado de Corpus Christi antecipa fim da campanha de vacinação contra a gripe em Viçosa

-----
<img src="https://s2.glbimg.com/S9_Btp_MvJcQFFvj3X5KTVUOU74=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/Q/n/l3G8VKTAuIdLuKXaZMjQ/26711939211-71d549ade1-k.jpg" /><br />   Vacinação está disponível até esta quarta-feira (30), das 8h às 17h, segundo Prefeitura. Crianças foram as que menos vacinaram contra a gripe em Viçosa, informou Prefeitura
Neto Talmeli/Divulgação
A campanha de va

Suggested: greve
Enter category or press <enter> to accept suggestion: automoveis
Category "automoveis" chosen

-----
Title:     'A Disneylândia de Saddam': como ditadores exploram ruínas históricas

-----
<img src="https://s2.glbimg.com/wnpwrJ7qgukdxIeHJNUAOs-7b94=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/l/r/YXJ7iaTPmRNnQlQLAwTw/1.jpg" /><br />   Seguindo exemplo de Mussolini e Hitler, ditador iraquiano se apropriou de simbolismo associado a ruínas históricas de seu país para evocar glórias passadas e alimentar culto a si mesmo. Convencido a estabelecer uma relação entre seu governo e os babilônios, Saddam Hussein encomendou este mural de si próprio numa carruagem
Getty Images
De tão reluzente, torna-se difícil mirar o palácio que se ergue em fachadas angulares e janelas abertas sob o intenso sol do Iraque. Uma curta estrada em espiral leva a seu topo, onde cascalhos soltos são cobertos por sombras de oliveiras e palmeiras que crescem livres no

Suggested: greve
Enter category or press <enter> to accept suggestion: Internacional
Category "internacional" chosen

-----
Title:     Greve de caminhoneiros: protestos causam impactos em diferentes setores no Paraná

-----
<img src="https://s2.glbimg.com/fiMrz0nsC7U1VZ7eSYCQePM5Yjc=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/n/J/TRoqIBQAmsWQ9OJpGqxw/598f6d4a-7b82-4499-a3a8-7e17eede2f7e.jpg" /><br />   Cidades estão sem combustível, aulas foram suspensas e há relatos de falta de alimentos.  Greve de caminhoneiros: protestos causam impactos em diferentes setores no Paraná Cidades estão sem combustível, aulas foram suspensas e há relatos de falta de alimentos.  Há 172 pontos de protestos nas estradas estaduais e 84 em rodovias federais. Universidades e escolas de todas as regiões suspendem aulas. Principais cidades estão sem combustíveis. No Paraná, caminhões com cargas essenciais serão liberados. Em Curitiba, venda de gás começa a ser normalizada
Su

...

Suggested: greve
Enter category or press <enter> to accept suggestion: moradia
Category "moradia" chosen

-----
Title:     Justiça determina desbloqueio de estradas municipais e estaduais de SC

-----
Já há decisões semelhantes da Justiça Federal. Caminhoneiros estão em greve há 9 dias. A Justiça atendeu a um pedido da Associação Nacional de Hospitais Privados (ANAHP) e determinou a desobstrução de rodovias estaduais e municipais de Santa Catarina. A greve dos caminhoneiros contra o preço do diesel chegou ao 9º dia. A determinação é da manhã desta terça-feira (29), dada pelo juiz Fernando de Castro Faria, e tem efeito em todo o estado. Já há decisões semelhantes da Justiça Federal. 
No despacho, o magistrado também autoriza o uso da força policial depois de 12 horas de negociações com os manifestantes. São réus na ação a Confederação Nacional dos Transportadores Autônomos (CNTA), o governo de Santa Catarina e demais pessoas físicas. A multa caso haja descumprimento é, respectivamente, 

Suggested: greve
Enter category or press <enter> to accept suggestion: 
Category "greve" chosen

-----
Title:     Juiz autoriza usina de SP a fornecer etanol diretamente para postos

-----
<img src="https://s2.glbimg.com/gCFmnunqpAt5bWvJsltmQlcOP08=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/o/m/w9aI1lS52OasUkPcRjLA/fila88.jpg" /><br />   Liminar também impede que a Agência Nacional do Petróleo aplique penalidades a produtora. Fila em posto de combustíveis em Embu das Artes
TV Globo/Reprodução
A Justiça Federal em São Paulo autorizou uma usina de Araçatuba, no interior de São Paulo, a fornecer etanol combustível diretamente para postos. A liminar também impede que a Agência Nacional do Petróleo, Gás Natural e Biocombustíveis (ANP) aplique penalidades. Cabe recurso.
Uma norma da ANP impede que as produtoras de etanol sejam também as fornecedoras do produto para os postos.
Por causa da greve dos caminhoneiros, que entrou nesta terça-feira (29) em seu

Suggested: greve
Enter category or press <enter> to accept suggestion: 
Category "greve" chosen

-----
Title:     VÍDEOS: MG Inter TV 1ª Edição, desta terça-feira, 29 de maio

-----
<img src="https://s2.glbimg.com/N2mvvi10uaw5IcoGri9iso7foEQ=/s01.video.glbimg.com/deo/vi/55/09/6770955" /><br />   Assista aos vídeos do telejornal com as notícias da Inter TV dos Vales. Assista aos vídeos do telejornal com as notícias da Inter TV dos Vales.
Suggested: estados
Enter category or press <enter> to accept suggestion: cidades
Category "cidades" chosen

-----
Title:     VÍDEOS: MG1 TV Integração Zona da Mata de terça-feira, 29 de maio

-----
<img src="https://s2.glbimg.com/IEtu_kj_faT3Bk-XFLUBAFaEkuU=/s01.video.glbimg.com/deo/vi/48/07/6770748" /><br />   Assista aos vídeos do telejornal com as notícias da Zona da Mata e Campo das Vertentes. Assista aos vídeos do telejornal com as notícias da Zona da Mata e Campo das Vertentes.
Suggested: estados
Enter category or press <enter> to accept suggestio

Suggested: greve
Enter category or press <enter> to accept suggestion: cidades
Category "cidades" chosen

-----
Title:     Jovem de 19 anos é encontrado degolado em bairro no interior do Acre

-----
Delegado acredita que crime foi motivado por acerto de contas. Corpo de José Paiva Saraiva foi encontrado por populares no bairro Senador Pompeu.  O jovem José Paiva Saraiva, de 19 anos, foi achado degolado na madrugada desta terça-feira (29), em uma rua no bairro Senador Pompeu, no município de Tarauacá, no interior do Acre. 
De acordo com informações da Polícia Civil, populares acionaram a polícia após encontrar o corpo do rapaz. O delegado que investiga o caso, José Obetâneo dos Santos, acredita que o crime foi motivado por um acerto de contas. 
Segundo ele, a vítima era integrante de uma facção criminosa e tinha passagens pela polícia pelos crimes de furto, roubo e tráfico de drogas. 
Até o início desta tarde ninguém foi preso e a polícia segue com as investigações para identificar e lo

Suggested: saude
Enter category or press <enter> to accept suggestion: 
Category "saude" chosen

List of categories stored in the database:
greve
estados
saude
automoveis
internacional
politica
policial
educacao
moradia
cidades


Do some tests now:

In [23]:
#cl.cprob(<word>,<category>)
print(cl.cprob('casa','moradia'))

0.2


In [24]:
#cl.fprob(<word>,<category>)
print(cl.fprob('casa','moradia'))

1.0
